# Stock Sentiment Analysis Prototype Using Finbert & TextBlob with FMP Data

In [2]:
# !pip install textblob

     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
      --------------------------------------- 10.2/636.8 kB ? eta -:--:--
     - ----------------------------------- 30.7/636.8 kB 262.6 kB/s eta 0:00:03
     ---- -------------------------------- 71.7/636.8 kB 393.8 kB/s eta 0:00:02
     ---------------- --------------------- 276.5/636.8 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 636.8/636.8 kB 2.5 MB/s eta 0:00:00
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)


In [8]:
# !pip install fmpsdk
# !pip install python-dotenv
# !pip install transformers

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     -------- ---------------------------- 30.7/129.4 kB 262.6 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/129.4 kB 409.6 kB/s eta 0:00:01
     ------------------------------------ 129.4/129.4 kB 692.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   -- ------------------------------------- 0.6/8.4 MB 12.4 MB/s eta 0:00:01
   -------- ------------------------------- 1.7/8.4 MB 18.1 MB/s eta 0:00:01
   --------------------------- ------------ 5.7/8.4 MB 40.6 MB/s eta 0:00:01
   ---------------------------------------  8.4/8.4 MB 48.8 MB/s eta 0:00:01
   ---------------------------------------- 8.4/8.4 MB 44.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/330.1 kB ? eta -:--:--
   --------------------------------------- 330.1/330.1 kB 20.0 MB/s eta 0:00:00
   -----

In [5]:
from textblob import TextBlob
import requests

In [2]:
# import requests
# from json.decoder import JSONDecodeError

# query = '"S&P 500" OR "SP500" OR "^GSPC"'

# # Your FMP API key
# api_key = 'Enter_Name'

# # Endpoint URL for the FMP stock news sentiments RSS feed
# url = f'https://financialmodelingprep.com/api/v4/stock-news-sentiments-rss-feed?page=0&apikey={api_key}'

# # Make the API request
# response = requests.get(url)

# # Check if the response status code is 200 (OK)
# if response.status_code == 200:
#     try:
#         # Parse the JSON response
#         data = response.json()
#         # Process the data...
#         print(data)  # Print the data for demonstration purposes
#     except JSONDecodeError:
#         print("JSON decoding failed. Response not in JSON format.")
#         print("Response text:", response.text)
# else:
#     print(f"Failed to fetch data: Status code {response.status_code}")
#     print("Response text:", response.text)

In [3]:
# print(response.text)

In [6]:
# Variables to track API usage
api_calls_made = 0
api_call_limit = 250

api_key = 'Enter_Name'  # Replace with your FMP API key

# Only proceed if we haven't reached the API call limit
if api_calls_made < api_call_limit:
    url = f'https://financialmodelingprep.com/api/v3/stock_news?tickers=SPY&limit=100&apikey={api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Increment the API call count
        api_calls_made += 1

        # Print the structure of 'data' to understand its format (for debugging)
        print(data)

        # Process the data
        for article in data:  # Assuming 'data' is a list of articles
            title = article.get('title', 'No Title')
            content = article.get('text', 'No Content')  # Adjusted to 'text' as per typical JSON response

            # Perform sentiment analysis
            sentiment_analysis = TextBlob(content)
            sentiment = sentiment_analysis.sentiment.polarity  # Value between -1 and 1

            # Output the results
            print(f"Title: {title}\nSentiment: {sentiment}\n")
    else:
        print(f"Failed to fetch data: Status code {response.status_code}")
else:
    print("API call limit reached for the day.")

[{'symbol': 'SPY', 'publishedDate': '2024-01-22 18:22:37', 'title': "Why Parametric's Equity Income ETF Looks Compelling", 'image': 'https://cdn.snapi.dev/images/v1/r/y/etf47-2239344.jpg', 'site': 'ETF Trends', 'text': 'Advisors often look to an equity income ETF during periods of uncertainty. With widespread uncertainty around interest rates, elections and changing policy, and the possibility of a recession, the stability offered by equity income funds may look particularly attractive right now.', 'url': 'https://www.etftrends.com/etf-yield-channel/why-parametrics-equity-income-etf-looks-compelling/'}, {'symbol': 'SPY', 'publishedDate': '2024-01-22 18:22:36', 'title': 'World Markets Watchlist: January 22, 2024', 'image': 'https://cdn.snapi.dev/images/v1/q/o/etf46-2239343.jpg', 'site': 'ETF Trends', 'text': "Two of the eight indexes on our world watch list have posted\xa0gains\xa0through January 22, 2024. Tokyo's Nikkei 225 finished in the top spot with a YTD gain of 9.21%.", 'url': 'h

In [6]:
import pandas as pd
import numpy as np
import requests

from dotenv import load_dotenv
import os
import fmpsdk as fmp

# Load environment variables
load_dotenv()

# Retrieve the API key from environment variables
fmp_key = os.getenv('Enter_Name')

# Fetch stock news for the S&P 500 ETF (ticker symbol "SPY")
news_sp500 = fmp.stock_news(apikey= fmp_key, tickers='SPY')

# Print the fetched news data
print(news_sp500)

{'Error Message': 'Special Endpoint : This endpoint is not available under your current subscription please visit our subscription page to upgrade your plan at https://site.financialmodelingprep.com/developer/docs/pricing'}


In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch

# Load the pre-trained FinBERT model
model_name = "yiyanghkust/finbert-tone"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def sentiment_analysis_finbert(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    positive, neutral, negative = predictions[0].tolist()
    return positive, neutral, negative

C:\Users\shane\anaconda3\envs\Python3-8-Build\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 533/533 [00:00<00:00, 180kB/s]
C:\Users\shane\anaconda3\envs\Python3-8-Build\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shane\.cache\huggingface\hub\models--yiyanghkust--finbert-tone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to act

In [21]:
import requests

# Variables to track API usage
api_calls_made = 0
api_call_limit = 250

api_key = 'Enter_Name'  # Replace with your FMP API key

# Only proceed if we haven't reached the API call limit
if api_calls_made < api_call_limit:
    url = f'https://financialmodelingprep.com/api/v3/stock_news?tickers=SPY&limit=100&apikey={api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Increment the API call count
        api_calls_made += 1

        # Process the data
        for article in data:
            title = article.get('title', 'No Title')
            content = article.get('text', 'No Content')
            published_date = article.get('publishedDate', 'No Date')

            # Perform sentiment analysis using FinBERT
            positive, neutral, negative = sentiment_analysis_finbert(content)

            # Output the results
            print(f"Date: {published_date}\nTitle: {title}\nPositive: {positive}, Neutral: {neutral}, Negative: {negative}\n")
    else:
        print(f"Failed to fetch data: Status code {response.status_code}")
else:
    print("API call limit reached for the day.")

Date: 2024-01-24 13:26:06
Title: Stocks May Pause: 5 Reasons to be "Strategically Inactive"
Positive: 0.0005955280503258109, Neutral: 0.9992669224739075, Negative: 0.0001375334250042215

Date: 2024-01-24 02:59:00
Title: Billionaires Can't Get Enough of This ETF in 2024. Is It Right for You?
Positive: 2.126578806382895e-07, Neutral: 0.9999995231628418, Negative: 2.185417145028623e-07

Date: 2024-01-23 12:02:03
Title: ETFs in Focus as S&P 500 Confirms Bull Market: How Long Will It Last?
Positive: 0.0009697534260340035, Neutral: 0.9835267663002014, Negative: 0.015503489412367344

Date: 2024-01-23 06:07:14
Title: Secondary squeeze: exploring 3 highly shorted stocks
Positive: 3.3839751267805696e-05, Neutral: 0.999961256980896, Negative: 4.917490969091887e-06

Date: 2024-01-22 18:22:37
Title: Why Parametric's Equity Income ETF Looks Compelling
Positive: 0.015632616356015205, Neutral: 0.4649147391319275, Negative: 0.5194526314735413

Date: 2024-01-22 18:22:36
Title: World Markets Watchlist: J

In [24]:
import csv

# Assuming 'sentiment_data' is a list of dictionaries containing your data
sentiment_data = []

for article in data:
    title = article.get('title', 'No Title')
    content = article.get('text', 'No Content')
    published_date = article.get('publishedDate', 'No Date')
    positive, neutral, negative = sentiment_analysis_finbert(content)
    sentiment_data.append({
        "date": published_date,
        "title": title,
        "positive": positive,
        "neutral": neutral,
        "negative": negative
    })

# Writing to a CSV file
keys = sentiment_data[0].keys()
with open('sentiment_data.csv', 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sentiment_data)

In [25]:
# Writing to a CSV file
keys = sentiment_data[0].keys()
with open('sentiment_data.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sentiment_data)